## XESMF with H5NetCDF, earthaccess, and pre-generated weights

Requires the upcoming ESMF 8.7 release for high-resolution datasets - https://github.com/pangeo-data/xESMF/issues/380

In [ ]:
import argparse
import itertools

import earthaccess
import numpy as np
import pyproj
import rasterio.transform
import xarray as xr
import xesmf as xe
from common import earthaccess_args

In [ ]:
def make_grid_ds() -> xr.Dataset:
    """
    Modified from ndpyramid - https://github.com/carbonplan/ndpyramid
    """
    dstSRS = "EPSG:3857"
    width = height = 256
    te = [
        -20037508.342789244,
        -20037508.342789244,
        20037508.342789244,
        20037508.342789244,
    ]

    transform = rasterio.transform.Affine.translation(
        te[0], te[3]
    ) * rasterio.transform.Affine.scale((te[2] * 2) / width, (te[1] * 2) / height)

    p = pyproj.Proj(dstSRS)

    grid_shape = (height, width)
    bounds_shape = (height + 1, width + 1)

    xs = np.empty(grid_shape)
    ys = np.empty(grid_shape)
    lat = np.empty(grid_shape)
    lon = np.empty(grid_shape)
    lat_b = np.zeros(bounds_shape)
    lon_b = np.zeros(bounds_shape)

    # calc grid cell center coordinates
    ii, jj = np.meshgrid(np.arange(height) + 0.5, np.arange(width) + 0.5)
    for i, j in itertools.product(range(grid_shape[0]), range(grid_shape[1])):
        locs = [ii[i, j], jj[i, j]]
        xs[i, j], ys[i, j] = transform * locs
        lon[i, j], lat[i, j] = p(xs[i, j], ys[i, j], inverse=True)

    # calc grid cell bounds
    iib, jjb = np.meshgrid(np.arange(height + 1), np.arange(width + 1))
    for i, j in itertools.product(range(bounds_shape[0]), range(bounds_shape[1])):
        locs = [iib[i, j], jjb[i, j]]
        x, y = transform * locs
        lon_b[i, j], lat_b[i, j] = p(x, y, inverse=True)

    return xr.Dataset(
        {
            "x": xr.DataArray(xs[0, :], dims=["x"]),
            "y": xr.DataArray(ys[:, 0], dims=["y"]),
            "lat": xr.DataArray(lat, dims=["y", "x"]),
            "lon": xr.DataArray(lon, dims=["y", "x"]),
            "lat_b": xr.DataArray(lat_b, dims=["y_b", "x_b"]),
            "lon_b": xr.DataArray(lon_b, dims=["y_b", "x_b"]),
        },
    )


def xesmf_weights_to_xarray(regridder) -> xr.Dataset:
    """
    From ndpyramid - https://github.com/carbonplan/ndpyramid
    """
    w = regridder.weights.data
    dim = "n_s"
    ds = xr.Dataset(
        {
            "S": (dim, w.data),
            "col": (dim, w.coords[1, :] + 1),
            "row": (dim, w.coords[0, :] + 1),
        }
    )
    ds.attrs = {"n_in": regridder.n_in, "n_out": regridder.n_out}
    return ds


def _reconstruct_xesmf_weights(ds_w):
    """
    Reconstruct weights into format that xESMF understands

    From ndpyramid - https://github.com/carbonplan/ndpyramid
    """
    import sparse
    import xarray as xr

    col = ds_w["col"].values - 1
    row = ds_w["row"].values - 1
    s = ds_w["S"].values
    n_out, n_in = ds_w.attrs["n_out"], ds_w.attrs["n_in"]
    crds = np.stack([row, col])
    return xr.DataArray(
        sparse.COO(crds, s, (n_out, n_in)), dims=("out_dim", "in_dim"), name="weights"
    )

In [ ]:
def reconstruct_weights(weights_fp):
    return _reconstruct_xesmf_weights(xr.open_zarr(weights_fp))

In [ ]:
def get_weights_fp(earthaccess_args):
    return (
        "s3://nasa-veda-scratch/test-weight-caching/"
        + earthaccess_args["filename"][:-4]
        + "_weights.zarr"
    )


def get_target_grid_fp(earthaccess_args):
    return (
        "s3://nasa-veda-scratch/test-weight-caching/"
        + earthaccess_args["filename"][:-4]
        + "_target.zarr"
    )

In [ ]:
def generate_weights(dataset):
    args = earthaccess_args[dataset]
    weights_fp = get_weights_fp(args)
    target_grid_fp = get_target_grid_fp(args)
    input_uri = f'{args["folder"]}/{args["filename"]}'
    src = f's3://{args["bucket"]}/{input_uri}'
    target_grid = make_grid_ds()
    target_grid.to_zarr(target_grid_fp, mode="w")
    fs = earthaccess.get_s3fs_session(daac=args["daac"])
    fsspec_caching = {
        "cache_type": "none",
    }
    with fs.open(src, **fsspec_caching) as f:
        da = xr.open_dataset(f, engine="h5netcdf", mask_and_scale=True)[
            args["variable"]
        ]
        regridder = xe.Regridder(
            da,
            target_grid,
            "nearest_s2d",
            periodic=True,
            extrap_method="nearest_s2d",
            ignore_degenerate=True,
        )
        weights = xesmf_weights_to_xarray(regridder)
        weights.to_zarr(weights_fp, mode="w")

In [ ]:
def regrid(dataset):
    args = earthaccess_args[dataset]
    weights_fp = get_weights_fp(args)
    target_grid_fp = get_target_grid_fp(args)
    weights = reconstruct_weights(weights_fp)
    grid = xr.open_zarr(target_grid_fp)
    input_uri = f'{args["folder"]}/{args["filename"]}'
    src = f's3://{args["bucket"]}/{input_uri}'
    fs = earthaccess.get_s3fs_session(daac=args["daac"])
    fsspec_caching = {
        "cache_type": "none",
    }
    with fs.open(src, **fsspec_caching) as f:
        da = xr.open_dataset(f, engine="h5netcdf", mask_and_scale=True)[
            args["variable"]
        ]
        regridder = xe.Regridder(
            da,
            grid,
            "nearest_s2d",
            periodic=True,
            extrap_method="nearest_s2d",
            ignore_degenerate=True,
            reuse_weights=True,
            weights=weights,
        )
        return regridder(da)

In [ ]:
if __name__ == "__main__":
    if "get_ipython" in dir():
        da = regrid("gpm_imerg")
    else:
        parser = argparse.ArgumentParser(description="Set environment for the script.")
        parser.add_argument(
            "--dataset",
            default="gpm_imerg",
            help="Dataset to resample.",
            choices=["gpm_imerg", "mursst"],
        )
        parser.add_argument(
            "--gen-weights",
            default=False,
            help="Generate weights",
            action=argparse.BooleanOptionalAction,
        )
        user_args = parser.parse_args()
        if user_args.gen_weights:
            generate_weights(user_args.dataset)
        da = regrid(user_args.dataset)